In [1]:
import sys
sys.path.append('..')

import inspect
import numpy as np
import pandas as pd
from scipy.optimize import curve_fit
from tqdm import tqdm

from src.data import conex_file_parser
from src.profile_functions import gaisser_hillas, gaisser_hillas_six, usp, anormal
from src.util import get_fit

Welcome to JupyROOT 6.26/10


### Configuration

In [2]:
input_files = '../data/conex/p_17-20eV_60deg/sibyll23d/*.root'
output_file = 'blablabla.csv.gz'
branches = ['Xdep', 'dEdX']
max_showers = 1000
fcn = usp # one of gaisser_hillas, gaisser_hillas_six, usp, anormal

# not intendend to be modified:
par_names = inspect.getfullargspec(fcn).args[1:]
err_names = [f'err_{s}' for s in par_names]
col_names = par_names + err_names + ['status', 'chi2', 'ndf']
npar = len(par_names)

### Fit showers

In [3]:
# the parser will allow to iterate over conex files
parser = conex_file_parser(input_files, branches, max_showers)

# a structure to hold fit data
data = np.zeros((max_showers, len(col_names)))

# loop over showers in the input files
for i, (x, y) in tqdm(enumerate(parser), total = max_showers):
  sig = 1e-2 * np.sqrt(y * np.sum(y))
  msk = sig/y <= 0.3
  par, err, sts, chi2 = get_fit(fcn, x[msk], y[msk], sig[msk])
  data[i, 0:npar] = par
  data[i, npar:2*npar] = err
  data[i, -3] = int(sts)
  data[i, -2] = chi2
  data[i, -1] = len(x) - npar

100%|██████████| 1000/1000 [00:00<00:00, 1398.50it/s]


### Put results in a data frame

In [4]:
df = pd.DataFrame(data, columns = col_names)
df

,nmax,xmax,l,r,err_nmax,err_xmax,err_l,err_r,status,chi2,ndf
0,1.837821e+07,842.320233,231.301287,0.239534,21209.784402,0.411533,0.189008,0.002014,1.0,0.842024,196.0
1,5.020788e+07,743.730802,231.783547,0.237732,48057.958361,0.346024,0.159164,0.001722,1.0,0.570963,196.0
2,6.664650e+05,801.385395,225.053412,0.265192,528.000588,0.274652,0.126248,0.001381,1.0,0.386121,196.0
3,1.385045e+07,783.229698,228.783908,0.225308,18223.759248,0.463629,0.212806,0.002288,1.0,1.084137,196.0
4,1.406811e+06,786.271352,242.123375,0.310758,3969.523185,1.045447,0.486068,0.004955,1.0,5.259502,196.0
...,...,...,...,...,...,...,...,...,...,...,...
995,5.402869e+06,797.921463,231.330186,0.225408,2341.765903,0.156767,0.072057,0.000782,1.0,0.116277,196.0
996,5.730999e+07,819.378660,240.605289,0.240415,56967.866703,0.373282,0.171821,0.001793,1.0,0.639473,196.0
997,5.850622e+07,893.801346,231.086184,0.203990,45457.417394,0.278056,0.127567,0.001365,1.0,0.378708,196.0
998,1.253698e+07,774.330655,231.425409,0.229080,12238.996503,0.349076,0.160051,0.001707,1.0,0.603266,196.0


### Save results

In [5]:
# df.to_csv(output_file)